In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

#import custom

# Root directory of the project
ROOT_DIR = "C:/Users/gauta/Downloads/mask_rcnn_polygon - main"

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")


WEIGHTS_PATH = "C:/Users/gauta/Downloads/mask_rcnn_polygon - main/mask_rcnn_object_0010.h5"

In [6]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 22 # Background + phone,laptop and mobile

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 80% confidence
    DETECTION_MIN_CONFIDENCE = 0.85

In [ ]:
config = CustomConfig()
CUSTOM_DIR = os.path.join(ROOT_DIR, "/dataset/")
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.75

config = InferenceConfig()
config.display()

In [8]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. 
        self.add_class("object", 1, "0")
        self.add_class("object", 2, "1")
        self.add_class("object", 3, "2")
        self.add_class("object", 4, "3")
        self.add_class("object", 5, "4")
        self.add_class("object", 6, "5")
        self.add_class("object", 7, "7")
        self.add_class("object", 8, "9")
        self.add_class("object", 9, "B")
        self.add_class("object", 10, "N")
        self.add_class("object", 11, "/")
        self.add_class("object", 12, ".")
        self.add_class("object", 13, "R")
        self.add_class("object", 14, "S")
        self.add_class("object", 15, "6")
        self.add_class("object", 16, "8")
        self.add_class("object", 17, "K")
        self.add_class("object", 18, "J")
        self.add_class("object", 19, "U")
        self.add_class("object", 20, "M")
        self.add_class("object", 21, "A")
        self.add_class("object", 22, "Y")
        
        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open('C:/Users/gauta/Downloads/mask_rcnn_polygon - main/dataset/train/DEMO_FINAL2_json.json'))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"0": 1,"1": 2,"2": 3,"3": 4,"4": 5,"5": 6,"7": 7,"9": 8,"B": 9,"N": 10,"/": 11,".": 12,"R": 13,"S": 14,"6": 15,"8": 16,"K": 17,"J": 18,"U": 19,"M": 20,"A": 21,"Y": 22} #,"xyz": 3}
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
# Inspect the model in training or inference modes values: 'inference' or 'training'
TEST_MODE = "inference"
ROOT_DIR = "C:/Users/gauta/Downloads/mask_rcnn_polygon - main/dataset"

def get_ax(rows=1, cols=1, size=16):
  """Return a Matplotlib Axes array to be used in all visualizations in the notebook.  Provide a central point to control graph sizes. Adjust the size attribute to control how big to render images"""
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
  return ax

# Load validation dataset
# Must call before using the dataset
CUSTOM_DIR = "C:/Users/gauta/Downloads/mask_rcnn_polygon - main/dataset"
dataset = CustomDataset()
dataset.load_custom(CUSTOM_DIR, "val")
dataset.prepare()

In [ ]:
config = CustomConfig()
#LOAD MODEL. Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

In [ ]:
# Load COCO weights Or, load the last model you trained
weights_path = WEIGHTS_PATH
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [ ]:
# Load the image using the dataset object
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

# Run object detection
results = model.detect([image], verbose=0)
# Get the ROIs and class ids for the selected image from the results object
r = results[0]
rois = r['rois']
class_ids = r['class_ids']

# Create a figure and axes
fig, ax = plt.subplots(1, figsize=(10, 10))

# Display the image
ax.imshow(image)

# Draw a green bounding box and write class name above each ROI
for i, roi in enumerate(rois):
    y1, x1, y2, x2 = roi
    class_name = dataset.class_names[class_ids[i]]
    label = f"{class_name}"
    ax.text(x1, y1, label, color='k', size=16,backgroundcolor=(0,1,0))
    rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor=(0, 1 , 0), facecolor='none')
    ax.add_patch(rect)

# Show the plot
plt.show()

In [ ]:
path_to_new_image = 'C:/Users/gauta/Downloads/mask_rcnn_polygon - main/dataset/test/Inspection02092022_161518683.bmp'
image = mpimg.imread(path_to_new_image)
print(f"Image shape: {image.shape}")

# Create a list of images with length equal to the model batch size
images = [image] * model.config.BATCH_SIZE

# Run object detection
results = model.detect(images, verbose=0)

# Get the ROIs and class ids for the selected image from the results object
r = results[0]
rois = r['rois']
class_ids = r['class_ids']

# Create a figure and axes
fig, ax = plt.subplots(1, figsize=(10, 10))

# Display the image
ax.imshow(image)
print(f"Image shape: {image.shape}")

# Draw a green bounding box and write class name above each ROI
for i, roi in enumerate(rois):
    y1, x1, y2, x2 = roi
    class_name = dataset.class_names[class_ids[i]]
    label = f"{class_name}"
    ax.text(x1, y1, label, color='k', size=16,backgroundcolor=(0,1,0))
    rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor=(0, 1 , 0), facecolor='none')
    ax.add_patch(rect)

# Show the plot
plt.show()


In [ ]:
#OCV
# Load the image using the dataset object
image, image_meta, gt_class_id, gt_bbox, gt_mask = \
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

# Run object detection
results = model.detect([image], verbose=0)

# Get the predicted classes as a string
predicted_classes = [class_names[class_id] for class_id in results[0]['class_ids']]
predicted_classes_str = ''.join(predicted_classes)
print('Predicted classes: ', predicted_classes_str)
input_string=("2/2N111229//4B./R002090.270125S3")


# Define the coordinates of the bounding box
height, width, _ = image.shape

# Define the coordinates of the bounding box
x1 = int(0.03 * width)  
y1 = int(0.3 * height)  
x2 = int(0.97 * width)  
y2 = int(0.7 * height)  
# Define the text to be displayed above the bounding box
if predicted_classes_str == input_string:
    text = 'PASS'
    bbox_color = (0, 1, 0)  # Green color
else:
    text = 'FAIL'
    bbox_color = (1, 0, 0)  # Red color

# Create a figure and axes
fig, ax = plt.subplots(1, figsize=(10, 10))

# Display the image
ax.imshow(image)

# Draw a bounding box around the region of interest
rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=4, edgecolor=bbox_color, facecolor='none')
ax.add_patch(rect)

# Add the text above the bounding box
ax.text((x1 + x2) / 2, y1 - 20, text, fontsize=24, color=bbox_color, ha='center', va='center')

# Show the plot
plt.show()